In [ ]:
!git clone https://github.com/dudegladiator/ResumeParser

In [ ]:
!pip install spacy==3.7.2

In [ ]:
from spacy.tokens import DocBin
from tqdm import tqdm
import random
import json
import spacy
from spacy.util import filter_spans
nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin()


!python -m spacy download en_core_web_lg

In [ ]:
###Train2

def trim_entity_spans(data: list) -> list:

    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data


def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines = []
        with open(dataturks_JSON_FilePath, 'r', encoding="utf8") as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            if data['annotation'] is not None:
                for annotation in data['annotation']:
                    # only a single point in text annotation.
                    point = annotation['points'][0]
                    labels = annotation['label']
                    # handle both list of labels or a single label.
                    if not isinstance(labels, list):
                        labels = [labels]

                    for label in labels:
                        # dataturks indices are both inclusive [start, end]
                        # but spacy is not [start, end)
                        entities.append((
                            point['start'],
                            point['end'] + 1,
                            label
                        ))

            training_data.append((text, {"entities": entities}))
        return training_data
    except Exception as e :

        logging.exception("Unable to process " + dataturks_JSON_FilePath)
        return None

# start with afreen dataset
TRAIN_DATA = trim_entity_spans(convert_dataturks_to_spacy("/content/model2_dataset.json"))



In [ ]:
training_data = []
for example in TRAIN_DATA:
  temp_dict = {}
  temp_dict['text'] = example[0]
  temp_dict['entities'] = []
  range1 = []
  for annotation in example[1]["entities"]:
    start = annotation[0]
    end = annotation[1]
    label = annotation[2].upper()
    skip = False
    for idx in range(start,end):
      if idx in range1:
        print("more --" ,idx)
        skip = True
        break
    if skip or start>=end:
          continue
    range1 += list(range(start,end))
    temp_dict['entities'].append((start, end, label))
  training_data.append(temp_dict)

print(training_data[0])

In [ ]:
count=0
for training_example  in tqdm(training_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="expand")
        if span is None:
            print("Skipping entity")

        else:
            ents.append(span)
    count+=1
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)
print(count)

doc_bin.to_disk("model2.spacy")

Please get the base config file from spacy website

In [ ]:
!python -m spacy init fill-config /kaggle/input/base-config/base_config.cfg config.cfg

If you get the config file, then directly run below code

---

Upload the config file to root


In [ ]:
!python -m spacy train /kaggle/input/config/config.cfg --output ./output2 --paths.train ./model2.spacy --paths.dev ./model2.spacy

go to the ouput2 folder, model-best is model2

In [ ]:
model2 = spacy.load("/content/output2/model-best")

In [ ]:
model2.get_pipe("ner").labels